<a href="https://colab.research.google.com/github/rakshit9/Generative-AI/blob/main/ThinkFlow_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip -q install "langchain>=0.3.0" "langgraph>=0.2.0" "langchain-openai>=0.2.0" "python-dotenv>=1.0.1"

In [44]:
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")




In [45]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI

In [46]:
# Define state
class State(dict):
  topic:str
  retrieved:str
  summary:str
  feedback:str

In [47]:
# Inititalize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [48]:
# NODE 1 : Retrieval
def retrieve_info(state: State):
  topic = state["topic"]
  state["retrieved"] = f"Top search results and notes about {topic}. (pretend web data)"
  print(f"Retrieved context for: {topic}")
  return state

In [49]:
def generate_summary(state: State):
    context = state["retrieved"]
    prompt = f"Summarize this information clearly and concisely:\n\n{context}"
    response = llm.invoke(prompt)
    state["summary"] = response.content
    print("Generated summary ✅")
    return state

In [50]:
# ----- Node 3: Evaluate Summary -----
def evaluate_summary(state: State):
    summary = state["summary"]
    prompt = f"Critique this summary for clarity and accuracy. Give a short feedback:\n\n{summary}"
    response = llm.invoke(prompt)
    state["feedback"] = response.content
    print("Evaluated summary 🔍")
    return state

In [51]:
# Added graph Node
graph = StateGraph(State)
graph.add_node("retrieve", retrieve_info)
graph.add_node("summarize", generate_summary)
graph.add_node("evaluate", evaluate_summary)

In [52]:
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "summarize")
graph.add_edge("summarize", "evaluate")
graph.set_finish_point("evaluate")

In [53]:
app = graph.compile()

In [54]:
# ----- Run the Agent -----
topic = "Impact of AI on healthcare"
result = app.invoke({"topic": topic})
print("\n--- FINAL OUTPUT ---")
print(f"Topic: {result['topic']}")
print(f"Retrieved: {result['retrieved']}")
print(f"Summary: {result['summary']}")
print(f"Feedback: {result['feedback']}")

Retrieved context for: Impact of AI on healthcare
Generated summary ✅
Evaluated summary 🔍

--- FINAL OUTPUT ---
Topic: Impact of AI on healthcare
Retrieved: Top search results and notes about Impact of AI on healthcare. (pretend web data)
Summary: Top search results show that AI is revolutionizing healthcare by improving diagnosis accuracy, treatment effectiveness, and patient outcomes. AI technologies such as machine learning and natural language processing are being used to analyze medical data, predict disease progression, and personalize treatment plans. However, concerns about data privacy, bias in algorithms, and job displacement are also being raised.
Feedback: This summary effectively highlights the positive impact of AI in healthcare, such as improving diagnosis accuracy and treatment effectiveness. It also mentions potential concerns like data privacy and bias in algorithms. However, it could benefit from providing more specific examples or statistics to further illustrate th